# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

# Notebook 08 — User Profile Instructions Generation (LLM-based)

## Goal
Demonstrate a **production-safe, explicit** mechanism that **generates** `UserProfile.system_instructions` using an LLM, based on:
- conversation history (user + assistant messages only),
- existing user profile (current instructions),
- (optionally) session metadata.

## Non-goals (explicitly out of scope)
- RAG / retrieval
- tools
- websearch
- chain-of-thought or hidden reasoning frameworks
- org-profile conflicts / precedence rules beyond baseline
- runtime modifications (no changes to `ask()` flow)

## Contract
This notebook introduces a **separate, explicit "profile consolidation" flow**:

history + existing profile
    → LLM
    → normalized instructions
    → persist to UserProfile.system_instructions
    → mark session as requiring refresh (`needs_user_instructions_refresh = True`)

Runtime remains unchanged and will only **consume** the refreshed instructions snapshot (baseline proven in Notebook 07).

## Source of truth
All code and structure must follow the Intergrax bundle:
- `INTERGRAX_ENGINE_BUNDLE.py`


## Conceptual Model — Profile Instructions Generation

This notebook separates **instruction generation** from **instruction usage**.

### Two distinct responsibilities

**Runtime (Notebook 07)**
- Uses `UserProfile.system_instructions`
- Caches them at session level
- Builds a runtime-only SYSTEM message
- Never generates or mutates instructions

**Profile Consolidation (Notebook 08)**
- Generates `UserProfile.system_instructions`
- Persists them to the user profile
- Explicitly marks sessions as requiring refresh
- Never participates in `ask()` execution

### Data flow (explicit and one-directional)

Conversation history (user + assistant only)
Existing UserProfile.system_instructions
(Optional) session metadata
        ↓
LLM-based consolidation
        ↓
Normalized system instructions (plain text)
        ↓
UserProfile.system_instructions (persisted)
        ↓
Session marked: needs_user_instructions_refresh = True

### Key invariants

- No system messages are read from or written to conversation history
- No runtime code calls the LLM to generate instructions
- No implicit refresh or auto-consolidation occurs
- Consolidation is an explicit, auditable operation

### Why this separation matters

- Runtime stays deterministic and debuggable
- Instruction generation can be tested, versioned, and audited independently
- The system matches ChatGPT-like behavior:
  instructions are **stable profile data**, not emergent runtime artifacts


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from __future__ import annotations

from dataclasses import dataclass
from typing import List, Optional, Protocol

from intergrax.llm.messages import ChatMessage
from intergrax.memory.stores.in_memory_user_profile_store import InMemoryUserProfileStore
from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager


# ---------------------------------------------------------------------
# Test identifiers
# ---------------------------------------------------------------------
USER_ID: str = "user_artur"
SESSION_ID: str = "sess_notebook08_001"


# ---------------------------------------------------------------------
# Minimal LLM contract used by the generation service in this notebook.
# (We keep it notebook-local and deterministic.)
# ---------------------------------------------------------------------
class InstructionsLLM(Protocol):
    async def generate_user_profile_instructions(
        self,
        *,
        existing_instructions: Optional[str],
        history: List[ChatMessage],
        metadata: Optional[dict] = None,
        max_chars: int = 800,
    ) -> str:
        ...


class FakeDeterministicInstructionsLLM:
    """
    Deterministic LLM stub for Notebook 08.

    Goal:
      - Test the generation + persistence + session refresh flow
      - Without relying on external APIs or model variability
    """

    async def generate_user_profile_instructions(
        self,
        *,
        existing_instructions: Optional[str],
        history: List[ChatMessage],
        metadata: Optional[dict] = None,
        max_chars: int = 800,
    ) -> str:
        # NOTE: We deliberately avoid "smart" behavior here.
        # We only reflect what Notebook 08 needs to validate.
        base = (
            "You are talking to Artur. "
            "Use a technical, concise style. "
            "Never use emojis in code blocks or technical documentation. "
            "Default project context: Intergrax nexus Runtime."
        )

        # A tiny, deterministic "history signal" to prove history was passed in.
        # We only count user turns (role == 'user') and include it in output.
        user_turns = 0
        for m in history:
            if m.role == "user":
                user_turns += 1

        suffix = f" (history_user_turns={user_turns})"
        out = (base + suffix).strip()

        if len(out) > max_chars:
            out = out[:max_chars].rstrip()

        return out


# ---------------------------------------------------------------------
# Notebook-local generation service
#   history -> LLM -> profile.system_instructions
#   + mark session needs_user_instructions_refresh=True
# ---------------------------------------------------------------------
@dataclass(frozen=True)
class GenerationResult:
    new_instructions: str
    history_user_turns: int


class UserProfileInstructionsGenerationService:
    def __init__(
        self,
        *,
        llm: InstructionsLLM,
        user_profile_store: InMemoryUserProfileStore,
        session_manager: SessionManager,
    ) -> None:
        self._llm = llm
        self._user_profile_store = user_profile_store
        self._session_manager = session_manager

    async def generate_and_persist(
        self,
        *,
        user_id: str,
        session_id: str,
        metadata: Optional[dict] = None,
        max_chars: int = 800,
    ) -> GenerationResult:
        # 1) Load current profile
        profile = await self._user_profile_store.get_profile(user_id)
        existing = profile.system_instructions

        # 2) Load conversation history (user+assistant only in baseline)
        history = await self._session_manager.get_history_for_session(session_id)

        # 3) Ask LLM to generate new instructions
        new_instructions = await self._llm.generate_user_profile_instructions(
            existing_instructions=existing,
            history=history,
            metadata=metadata,
            max_chars=max_chars,
        )

        # 4) Persist to profile (domain data after generation)
        profile.system_instructions = new_instructions
        await self._user_profile_store.save_profile(profile)

        # 5) Mark session refresh required (so runtime will re-snapshot later)
        session = await self._session_manager.get_session(session_id)
        if session is not None:
            session.needs_user_instructions_refresh = True
            await self._session_manager.save_session(session)

        # For assertions / debugging
        user_turns = 0
        for m in history:
            if m.role == "user":
                user_turns += 1

        return GenerationResult(new_instructions=new_instructions, history_user_turns=user_turns)


# ---------------------------------------------------------------------
# In-memory infrastructure (stores + manager)
# ---------------------------------------------------------------------
session_storage = InMemorySessionStorage(max_history_messages=50)
session_manager = SessionManager(storage=session_storage)

user_profile_store = InMemoryUserProfileStore()

llm = FakeDeterministicInstructionsLLM()
generation_service = UserProfileInstructionsGenerationService(
    llm=llm,
    user_profile_store=user_profile_store,
    session_manager=session_manager,
)


# ---------------------------------------------------------------------
# Seed: create session + seed conversation history + seed an initial profile
# ---------------------------------------------------------------------
async def _seed() -> None:
    # Create session
    await session_manager.create_session(session_id=SESSION_ID, user_id=USER_ID)

    # Seed baseline-style history (user/assistant only)
    seed_messages: List[ChatMessage] = [
        ChatMessage(role="user", content="We are working on Intergrax nexus Runtime."),
        ChatMessage(role="assistant", content="Understood. I will keep it technical and concise."),
        ChatMessage(role="user", content="Never use emojis in code or technical docs."),
        ChatMessage(role="assistant", content="Acknowledged."),
    ]

    for msg in seed_messages:
        await session_manager.append_message(SESSION_ID, msg)

    # Seed initial profile instructions (simulate a pre-existing state)
    profile = await user_profile_store.get_profile(USER_ID)
    profile.system_instructions = (
        "You are talking to Artur. Use a technical, concise style."
    )
    await user_profile_store.save_profile(profile)


await _seed()

print("CONFIG READY")
print(f"- USER_ID: {USER_ID}")
print(f"- SESSION_ID: {SESSION_ID}")
print("- Seeded profile.system_instructions and baseline-style history")



CONFIG READY
- USER_ID: user_artur
- SESSION_ID: sess_notebook08_001
- Seeded profile.system_instructions and baseline-style history


# Cell 4 — Execute generation and validate persistence + session refresh
#
# What we validate in this cell:
# 1) LLM-based instructions were generated using the session history
# 2) UserProfile.system_instructions was updated (persisted)
# 3) ChatSession.needs_user_instructions_refresh == True

In [2]:
async def _run_and_validate() -> None:
    # --- Before snapshot ---
    before_profile = await user_profile_store.get_profile(USER_ID)
    before_session = await session_manager.get_session(SESSION_ID)

    before_instr = before_profile.system_instructions
    before_refresh_flag = before_session.needs_user_instructions_refresh if before_session else None

    # --- Run generation ---
    result = await generation_service.generate_and_persist(
        user_id=USER_ID,
        session_id=SESSION_ID,
        metadata={"notebook": "08", "purpose": "instructions_generation"},
        max_chars=800,
    )

    # --- After snapshot ---
    after_profile = await user_profile_store.get_profile(USER_ID)
    after_session = await session_manager.get_session(SESSION_ID)

    after_instr = after_profile.system_instructions
    after_refresh_flag = after_session.needs_user_instructions_refresh if after_session else None

    # --- Assertions (hard checks) ---
    assert after_instr is not None and after_instr.strip(), "Expected non-empty generated instructions."
    assert after_instr != (before_instr or ""), "Expected instructions to change after generation."
    assert after_refresh_flag is True, "Expected session.needs_user_instructions_refresh to be True."

    # We also validate that history was actually considered (our stub encodes it deterministically).
    expected_suffix = f"(history_user_turns={result.history_user_turns})"
    assert expected_suffix in after_instr, "Expected generated instructions to include the history signal."

    # --- Print summary ---
    print("GENERATION OK")
    print(f"- before_instructions: {before_instr!r}")
    print(f"- after_instructions : {after_instr!r}")
    print(f"- before_refresh_flag: {before_refresh_flag}")
    print(f"- after_refresh_flag : {after_refresh_flag}")
    print(f"- history_user_turns : {result.history_user_turns}")


await _run_and_validate()

GENERATION OK
- before_instructions: 'You are talking to Artur. Use a technical, concise style.'
- after_instructions : 'You are talking to Artur. Use a technical, concise style. Never use emojis in code blocks or technical documentation. Default project context: Intergrax nexus Runtime. (history_user_turns=2)'
- before_refresh_flag: False
- after_refresh_flag : True
- history_user_turns : 2


# Cell 6 — Explicit session refresh and snapshot validation
#
# Purpose:
# Prove that after consolidation:
# - the session is refreshed,
# - a new user_profile_instructions snapshot is available at session level,
# - WITHOUT calling runtime ask() or mutating conversation history.
#
# This mirrors the baseline behavior proven in Notebook 07.

In [5]:
from intergrax.memory.user_profile_manager import UserProfileManager

async def _refresh_and_validate_snapshot() -> None:
    # Load session
    session_before = await session_manager.get_session(SESSION_ID)
    assert session_before is not None, "Session not found."

    # Ensure SessionManager can resolve user-level instructions.
    # get_user_profile_instructions_for_session() requires a configured UserProfileManager.
    effective_session_manager = session_manager
    if effective_session_manager._user_profile_manager is None:
        effective_session_manager = SessionManager(
            storage=session_storage,
            user_profile_manager=UserProfileManager(user_profile_store),
        )

    # Force refresh request (do NOT assume flag state from previous cells).
    session_before.needs_user_instructions_refresh = True
    await effective_session_manager.save_session(session_before)

    # Run "lazy refresh" path:
    # - resolve from UserProfileManager.get_system_instructions_for_user(user_id)
    # - cache on session.user_profile_instructions
    # - clear needs_user_instructions_refresh
    refreshed = await effective_session_manager.get_user_profile_instructions_for_session(
        session_before
    )
    assert isinstance(refreshed, str) and refreshed.strip(), (
        "Expected a non-empty instructions string after refresh."
    )

    # Post-refresh snapshots
    session_after = await effective_session_manager.get_session(SESSION_ID)
    assert session_after is not None, "Session not found after refresh."

    profile_after = await user_profile_store.get_profile(USER_ID)

    # Assertions (snapshot + flags)
    assert session_after.needs_user_instructions_refresh is False, (
        "Expected refresh flag to be cleared after snapshot refresh."
    )
    assert session_after.user_profile_instructions == profile_after.system_instructions, (
        "Expected session snapshot to match persisted UserProfile.system_instructions."
    )

    # Ensure history integrity (history is stored in SessionStorage, not on ChatSession)
    messages = await effective_session_manager.get_history_for_session(session_id=SESSION_ID)
    roles = [m.role for m in messages]
    assert all(r in ("user", "assistant") for r in roles), (
        "History integrity violated: only user/assistant roles must be persisted."
    )

    # Output
    print("REFRESH OK")
    print("- session snapshot updated via get_user_profile_instructions_for_session(session)")
    print("- refresh flag cleared")
    print("- history unchanged (user/assistant only)")
    print("- snapshot value:")
    print(session_after.user_profile_instructions)


await _refresh_and_validate_snapshot()


REFRESH OK
- session snapshot updated via get_user_profile_instructions_for_session(session)
- refresh flag cleared
- history unchanged (user/assistant only)
- snapshot value:
You are talking to Artur. Use a technical, concise style. Never use emojis in code blocks or technical documentation. Default project context: Intergrax nexus Runtime. (history_user_turns=2)


## Notebook 08 — Final Summary

This notebook demonstrated a **complete, production-safe flow** for generating
`UserProfile.system_instructions` using an LLM, while preserving a strict
separation of responsibilities across the Intergrax architecture.

### What was implemented and validated

1) **Explicit profile consolidation flow**
- Conversation history (user + assistant only) is used as input
- Existing profile instructions are included as context
- An LLM generates normalized system instructions
- The result is persisted as domain data on `UserProfile`

2) **Session refresh contract**
- The session is explicitly marked with `needs_user_instructions_refresh = True`
- Refresh happens lazily via `SessionManager.get_user_profile_instructions_for_session(session)`
- A new per-session snapshot is cached and used by runtime
- The refresh flag is cleared automatically

3) **Runtime isolation preserved**
- Runtime code was not modified
- No LLM calls occur inside `ask()`
- No system messages are persisted to conversation history
- Instruction precedence rules remain unchanged

### What this notebook did NOT do

- No RAG
- No tools
- No websearch
- No chain-of-thought
- No org-profile conflicts
- No runtime heuristics or feature flags

### Architectural guarantees

- Instruction generation is **auditable, testable, and deterministic**
- Runtime behavior remains **clean, minimal, and predictable**
- The system behaves in a **ChatGPT-like** manner:
  instructions are stable profile data, not emergent runtime artifacts

### Next logical steps (outside this notebook)

- Conflict resolution: user vs organization instructions
- Scheduling and cooldown policies for consolidation
- Multi-pass or layered instruction generation
- Optional RAG-assisted enrichment of profile instructions
